In [1]:
import torch
import torchvision
# neural netwrok modules
import torch.nn as nn

# Optimization functions
import torch.optim as optim

# functions without parameters
import torch.nn.functional as F

# dataset management
from torch.utils.data import DataLoader

# standard datasets
import torchvision.datasets as datasets

# transformations on dataset
import torchvision.transforms as transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# Hyperparameters
num_classes = 10
learning_rate = 1e-3 # 10^-3
batch_size = 1024
num_epochs = 5

In [18]:
# load pretrained model
model = torchvision.models.vgg16(pretrained=False)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [4]:
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [8]:
# input pass without change
# to test modification
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

In [19]:
# modify the pretrained models
# if finetuning, set requires_grad = False
for param in model.parameters():
    param.requires_grad = False
    
model.avgpool = Identity()
model.classifier = nn.Sequential(
    nn.Linear(512, 100),
    nn.ReLU(),
    nn.Linear(100, num_classes)
)
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [11]:
# load data
train_dataset = datasets.CIFAR10(root="dataset/", train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

100.0%


Extracting dataset/cifar-10-python.tar.gz to dataset/


In [12]:
# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [22]:
# Train networks
for epoch in range(num_epochs):
    losses = []
    print(epoch, "start")
    for batch_idx, (data, targets) in enumerate(train_loader):
        # get data to cuda
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        # forward
        pred = model(data)
        loss = criterion(pred, targets)
        
        losses.append(loss.item())
        
        # backward
        optimizer.zero_grad()
        loss.backward()
        
        # gradient descent or adam step
        optimizer.step()
        
        print(f"Cost at epoch {epoch} is {sum(losses)/len(losses):.5f}")
        
        

0 start
Cost at epoch 0 is 2.30185
Cost at epoch 0 is 2.30442
Cost at epoch 0 is 2.30374
Cost at epoch 0 is 2.30349
Cost at epoch 0 is 2.30347
Cost at epoch 0 is 2.30357
Cost at epoch 0 is 2.30332
Cost at epoch 0 is 2.30353
Cost at epoch 0 is 2.30355
Cost at epoch 0 is 2.30348
Cost at epoch 0 is 2.30352
Cost at epoch 0 is 2.30363
Cost at epoch 0 is 2.30365
Cost at epoch 0 is 2.30373
Cost at epoch 0 is 2.30372
Cost at epoch 0 is 2.30370
Cost at epoch 0 is 2.30377
Cost at epoch 0 is 2.30368
Cost at epoch 0 is 2.30376
Cost at epoch 0 is 2.30369
Cost at epoch 0 is 2.30382
Cost at epoch 0 is 2.30379
Cost at epoch 0 is 2.30382
Cost at epoch 0 is 2.30382
Cost at epoch 0 is 2.30374
Cost at epoch 0 is 2.30367
Cost at epoch 0 is 2.30365
Cost at epoch 0 is 2.30375
Cost at epoch 0 is 2.30378
Cost at epoch 0 is 2.30384
Cost at epoch 0 is 2.30396
Cost at epoch 0 is 2.30396
Cost at epoch 0 is 2.30389
Cost at epoch 0 is 2.30389
Cost at epoch 0 is 2.30392
Cost at epoch 0 is 2.30396
Cost at epoch 0 is 2

KeyboardInterrupt: 

In [16]:
# check accuracy on training & test
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("checking accuracy on training data")
    else:
        print("checking accuracy on testing data")
    
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            
            score = model(x)
            _, predict = score.max(1)
            num_correct += (predict == y).sum()
            num_samples += predict.size(0)
            
        print(f"Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}")
    model.train()

In [17]:
check_accuracy(train_loader, model)

checking accuracy on training data


KeyboardInterrupt: 